In [14]:
import numpy as np
import unittest
import weakref
from memory_profiler import profile

In [15]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [16]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
                
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x:x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            #print(f.outputs[0].data)
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    #x.grad += gx
                if x.creator is not None:
                    add_func(x.creator)

    def cleargrad(self):
        self.grad = None

In [17]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        self.generation = max([x.generation for x in inputs])
        for output in outputs: 
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs)>1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()
    
class Add(Function):
    
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy
    
class Square(Function):
    
    def forward(self, x):
        return x**2
    
    def backward(self, gy):
        x = self.inputs[0].data
        return 2*x*gy
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.inputs[0].data
        return np.exp(x)*gy
    
def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [26]:
%load_ext memory_profiler
%%memray_flamegraph

for i in range(10):
    x = Variable(np.random.randn(10000))
    y = square(square(square(x)))

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


UsageError: Line magic function `%%memray_flamegraph` not found.


In [25]:
%mprun -f Lloop Lloop(10)

ERROR: Could not find file C:\Users\shuta\AppData\Local\Temp\ipykernel_10212\1719085581.py



In [10]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0


In [10]:
generations = [2, 0 ,1, 4, 2, 1]
funcs = []
for g in generations:
    f = Function()
    f.generation = g
    funcs.append(f)

funcs.sort(key=lambda x:x.generation)
[f.generation for f in funcs]

[0, 1, 1, 2, 2, 4]

In [11]:
s = set()
s.add(1)
s.add(2)
s.add(3)
s.add(1)
s

{1, 2, 3}

In [62]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print("y.grad: {} ({})".format(y.grad, id(y.grad)))
print("x.grad: {} ({})".format(x.grad, id(x.grad)))

gys  3120459274928
gxs[0] 3120459274928
gxs[1] 3120459274928
y.grad: 1.0 (3120459274928)
x.grad: 2.0 (3120459105808)


In [49]:

y = add(add(x, x), x)
y.backward()
print(x.grad)

x.cleargrad()
print("clear")
print(x.grad)
y.backward() #x.gradはNoneにしたけどyは残っている
print(x.grad)

x.cleargrad()
print("clear")
y = add(add(x, x), x)
y.backward()
print(x.grad)

y.grad: 2.0 (3120427316496)
x.grad: 2.0 (3120427316496)
5.0
clear
None
8.0
clear
4.0


In [41]:
x0 = Variable(np.array(2.))
x1 = Variable(np.array(3.))
y = add(square(x0), exp(x1))
y.backward()
print(x0.grad)
print(x1.grad)

3120459162944
3120459162944
3120458961216
3120459198208
3120459198208
3120459107888
3120459067008
3120459067008
3120459107504
4.0
20.085536923187668


In [27]:
x = Variable(np.array(0.5))

a = square(x)
b = exp(a)
y = square(b)

y.backward()
print(x.grad)

1.648721270700128
1.2840254166877414
0.25
3.297442541400256


In [11]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data-y0.data)/2./eps

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
    def test_grad_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(y.creator, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
    
# TestSuiteインスタンスを作成する
suite = unittest.TestSuite()

# TestCase派生クラスをテストスイートに追加する
suite.addTest(unittest.makeSuite(SquareTest))

# テストケース単体をテストスイートに追加する
#suite.addTest(SquareTest('test_forward'))

# TextTestRunner でテストスイートを実行する
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.033s

OK


3.0
None
1
[0.91439732]
None
1


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [10]:
data = np.array(1.0)
x = Variable(data)
x.data

array(1.)

In [8]:
print(x.data.ndim)
print(np.array([1, 2, 3]).ndim)

0
1


In [13]:
x = Variable(np.array(2))
f = Function()
y = f(x)
y.data

4

In [ ]:
"""    
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    return (f(x1).data - f(x0).data) / 2. / eps

def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return A(B(C(x)))
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
dy

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)
"""
"""
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A

y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)
"""

In [23]:
l = [1, 2, 3, 4]
while l:
    l.pop()
    print(l)
print(bool(l))

l.append(1)
print(l)
l.append(2)
l

[1, 2, 3]
[1, 2]
[1]
[]
False
[1]


[1, 2]

In [76]:
print(np.ones_like(1.0))
print(np.ones_like([1, 2]))


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [27]:
np.isscalar(as_array(1))

False

In [14]:
def f(*x):
    print(x)
    return x

type(f(1, 2, 3))

(1, 2, 3)


tuple

In [15]:
x = [1, 2, 3]
x**2

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [41]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(2.0))
add(x0, x1).data

array(3.)

In [44]:
names = ['Taro', 'Picasso', 'Cezanne']
ages =  [84, 91, 67]

for name, age in zip(names, ages):
    print(name, age)
print(type(zip(names, ages)))

Taro 84
Picasso 91
Cezanne 67
<class 'zip'>


In [4]:
set_test = set()
print(type(set_test))
set_test = {1, 2, 3, 2, 3}
print(set_test)
set_test.add(4)
set_test.add(3)
set_test

<class 'set'>
{1, 2, 3}


{1, 2, 3, 4}

In [ ]:
gen = [1, 2, 3, 4, 5, 2]
funcs = []

for g in gen:
    f = Function()
    f.generation = g
    funcs.append(f)
    

# 再帰とループ処理

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. Get a function
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward
            x.backward()



In [ ]:

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # Set parent(function)
        self.input = input
        self.output = output  # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx




In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [9]:
#再帰
def recursive(n):
    if n==0:
        return 1
    else:
        print(n, "*", "recursive(", n-1, ") = ", n*recursive(n-1))
        return n*recursive(n-1)

# 末尾再帰
def fact(n, r = 1):
    if n == 0:
        return r
    else:
        print("fact(", n-1, ", ", n*r, ")")
        return fact(n - 1, n * r)


In [13]:
fact(3)

fact( 2 ,  3 )
fact( 1 ,  6 )
fact( 0 ,  6 )


6

In [12]:
recursive(3)

1 * recursive( 0 ) =  1
2 * recursive( 1 ) =  2
1 * recursive( 0 ) =  1
3 * recursive( 2 ) =  6
1 * recursive( 0 ) =  1
2 * recursive( 1 ) =  2
1 * recursive( 0 ) =  1


6

In [17]:
x = np.array([1.0])
y = x**2
print(type(x), x.ndim)
print(type(y), y.ndim)

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'> 1


In [18]:
x = np.array(1.0)
y = x**2
print(type(x), x.ndim)
print(type(y), y.ndim)

<class 'numpy.ndarray'> 0
<class 'numpy.float64'> 0


In [2]:
1e1

10.0

In [13]:
N, A, B = map(int, input().split())
sum_part = 0
sum = 0
n4 = 0
n3 = 0
n2 = 0
n1 = 0
n0 = 0

for i in range(1, N+1, 1):
    n4 = int(i/1e4)
    n3 = int(i/1e3) - n4*1e1
    n2 = int(i/1e2) - n4*1e2 -n3*1e1
    n1 = int(i/1e1) - n4*1e3 - n3*1e2 - n2*1e1
    n0 = i - n4*1e4 - n3*1e3 - n2*1e2 - n1*1e1
    sum_part = n4 + n3 + n2 + n1 + n0
    if sum_part>=A and sum_part<=B:
        sum += i

print(sum)
  

 10 1 2


13


In [22]:
a = list(map(int, input().split()))

 2 3


In [24]:
a[1]

3

In [ ]:
N = int(input())
a = list(map(int, input().split()))
a.sort(reverse=True)

Alice = 0
Alice_counter = 0
Bob = 0

for i in range(N):
    if(Alice_counter==0):
        Alice += a[i]
        Alice_counter = 1
    else:
        Bob += a[i]
        Alice_counter = 0

1 2
 10 3


7


In [32]:
for _ in range(4):
    print("a")

a
a
a
a


# 付録A in-place 演算

In [29]:
x = np.array(1)
print(id(x))
x += x
print(id(x))
print(x)
x = x+x
print(id(x))

3120425903856
3120425903856
2
3120405281808


In [31]:
x = np.array(1)
z = x
print("x_before: ", id(x))
print("z_before: ", id(z))
z = x + x
print("x = ", x)
print("z = ", z)
print("x_after : ", id(x))
print("z_after : ", id(z))


x_before:  3120436084240
z_before:  3120436084240
x =  1
z =  2
x_after :  3120436084240
z_after :  3120405281808


In [20]:
x = np.array(1)
z = x
print("x_before: ", id(x))
print("z_before: ", id(z))
z = x + x
print("x_after : ", id(x))
print("z_after : ", id(z))
z

x_before:  3120436523632
z_before:  3120436523632
x_after :  3120436523632
z_after :  3120436392176


2

In [22]:
x = np.array(1)
y = np.array(2)
print("x_before: ", id(x))
print("y_before: ", id(y))
x += y
print("x_after : ", id(x))
print("y_after : ", id(y))

x_before:  3120425906064
y_before:  3120425903856
x_after :  3120425906064
y_after :  3120425903856


In [23]:
x = np.array(1)
y = np.array(2)
print("x_before: ", id(x))
print("y_before: ", id(y))
x += x
print("x_after : ", id(x))
print("y_after : ", id(y))

x_before:  3120427318608
y_before:  3120425906064
x_after :  3120427318608
y_after :  3120425906064


In [1]:
import weakref
import numpy as np
a = np.array([1, 2, 3])
b = weakref.ref(a)
a = None
print(b())
A = np.array(1)
B = A
A = None
print(B)
b

None
1


<weakref at 0x000001B908828E00; dead>

In [4]:
import numpy as np
np.ones((3, 2, 1))

array([[[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]]])

In [8]:
import contextlib
@contextlib.contextmanager
def config_test():
    print("start")
    try:
        yield
    finally:
        print("done")

with config_test():
    print("process..")

start
process..
done


In [9]:
@contextlib.contextmanager
def tag(name):
    print("{} start".format(name))
    yield
    print('{} end'.format(name))


@tag('decorated')
def f(content):
    print(content)


if __name__ == '__main__':
    f('this is test message')

decorated start
this is test message
decorated end


In [10]:
@contextlib.contextmanager
def tag(name):
    print("{} start".format(name))
    yield
    print('{} end'.format(name))


if __name__ == '__main__':
    with tag('decorated'):
        print('this is test message')

decorated start
this is test message
decorated end


In [13]:
z = np.array(1)
z.__array_priority__

0.0